# Imports and Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import pickle
import os
from utils import Config, Training_config

main_args = Config()

In [3]:
!pip install simpletransformers

In [4]:
from simpletransformers.ner import NERModel

In [5]:
output_base_dir = os.path.join(os.path.abspath('.'), 'outputs')

args = Training_config(main_args.ECC_TRAIN_CONFIG)
labels = args.BIO_labels

if args.model_type != 'interactive':
    labels = labels[:-1]

train_file, eval_file, test_file = args.train_dev_data

print("reading train file:", train_file)
with open(train_file, 'rb') as f:
    train_data = pickle.load(f)
print("reading eval file:", eval_file)
with open(eval_file, 'rb') as f:
    eval_data = pickle.load(f)
print("reading test file:", test_file)
with open(test_file, 'rb') as f:
    test_data = pickle.load(f)

if args.debug:
    train_data = train_data[:[i for i, d in enumerate(train_data) if d[0]<6][-1]]
    eval_data = eval_data[:[i for i, d in enumerate(eval_data) if d[0]<6][-1]]
    test_data = test_data[:[i for i, d in enumerate(test_data) if d[0]<6][-1]]
    wandb_project = False
else:
    wandb_project = "gec_ner"

columns=["sentence_id", "words", "labels", "cls_labels", "correction_index", "parsing_embedding"]

train_df = pd.DataFrame(train_data, columns=columns)
eval_df = pd.DataFrame(eval_data, columns=columns)
if test_file:
    test_df = pd.DataFrame(test_data, columns=columns)
if not args.parsing_embedding:
    train_df = train_df.drop(['parsing_embedding'], axis=1)
    eval_df = eval_df.drop(['parsing_embedding'], axis=1)
    test_df = test_df.drop(['parsing_embedding'], axis=1)

print(len(train_df), len(eval_df))
print(len(test_df))

if args.only_inference is not None:
    args.model_name = output_base_dir + args.exp_name
print(args.exp_name)

if args.only_inference is not None:
    if args.output_dir is None:
        output_dir = output_base_dir + args.exp_name
    else:
        output_dir = args.output_dir
        args.exp_name = output_dir
else:
    output_dir = output_base_dir + args.exp_name + '/eval'

model_args = {"overwrite_output_dir": True,
          "num_train_epochs": args.epochs,
          "train_batch_size": args.train_batch_size,
          "eval_batch_size": args.eval_batch_size,
          "output_dir": output_dir,
          "reprocess_input_data": True,
          "special_tokens_list": ["[NONE]", "[MOD]"],
          "wandb_kwargs": {
              "mode": 'online', #'offline',
              "name": args.exp_name,
          },
          "wandb_project": wandb_project,
          "evaluate_during_training": args.evaluate_during_training,
          "evaluate_each_epoch": args.evaluate_each_epoch,
          "learning_rate": args.lr,
          "multi_loss": args.multi_loss,
          "wo_token_labels": args.wo_token_labels,
          "cls_num_labels": 15, # label nums for [CLS] token classification
          "use_multiprocessing_for_evaluation": False,
          "use_multiprocessing": args.use_multiprocessing,
          "loss_weight": args.loss_weight,
          "max_correction_embeddings": args.max_correction_embeddings,
          "max_seq_length": args.max_seq_length,
          "n_gpu": args.n_gpu,
          "dataloader_num_workers": 20,
          "save_eval_checkpoints": False,
          "early_stopping_metric": "f1_score",
          "best_model_dir": output_dir,
          "parsing_embedding": args.parsing_embedding,
          "parsing_embedding_for_embedding": args.parsing_embedding_for_embedding,
          "logging_steps": 0,
          "manual_seed": 42
          }

model = NERModel(
    model_type=args.model,
    model_name=args.model_name,
    labels=labels,
    args=model_args,
    use_cuda=args.use_cuda
)

# Create a NERModel
if args.only_inference is None:
    # Train the model
    model.train_model(train_df, eval_data=eval_df, test_data=test_df)

if args.only_inference is None or 'dev' in args.only_inference:
    result, model_outputs, predictions, out_label_list = model.eval_model(eval_df, wandb_log=False)
if args.only_inference is None or 'test' in args.only_inference:
    result, model_outputs, predictions, out_label_list = model.eval_model(test_df, wandb_log=False)

reading train file: processed_data/bert_train.pkl
reading eval file: processed_data/bert_dev.pkl
reading test file: processed_data/bert_test.pkl
130210 16164
16497
/error_correction_ce_syntax/ner_1e5_bs32_bert_large


Some weights of BertForTokenClassification were not initialized from the model checkpoint at espejelomar/beto-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/4152 [00:00<?, ?it/s]

/Users/rams/miniforge3/envs/env_arm_py39/lib/python3.9/site-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

wandb: Currently logged in as: ram-senth (langbot). Use `wandb login --relogin` to force relogin


/Users/rams/miniforge3/envs/env_arm_py39/lib/python3.9/site-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Running Epoch 0 of 10:   0%|          | 0/519 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x1629db0d0>
Traceback (most recent call last):
  File "/Users/rams/miniforge3/envs/env_arm_py39/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/Users/rams/miniforge3/envs/env_arm_py39/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1474, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/Users/rams/miniforge3/envs/env_arm_py39/lib/python3.9/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/Users/rams/miniforge3/envs/env_arm_py39/lib/python3.9/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/Users/rams/miniforge3/envs/env_arm_py39/lib/python3.9/multiprocessing/connection.py", line 936, in wait
    ready = selector.select(timeout)
  File "/Users/rams/miniforge3/envs/env_arm_py39/lib/python3.9/selectors